In [1]:
import glob
import pandas as pd
import boto3
from datetime import datetime, timedelta
import json
import numpy as np
from utils.ingest_utils import generator_to_np, convert_time, get_col_index_map

In [2]:
file_cond = glob.glob('/Users/Indhuja/Desktop/tcsp/lip/TCSP_LIP_cond_2005.187_050706_1948.txt')[0]
file_fm = glob.glob('/Users/Indhuja/Desktop/tcsp/lip/TCSP_LIP_fieldmill_2005.187_050706.txt')[0]

df1 = pd.read_csv(file_cond, sep=r"\s+", header=None, skiprows=1, usecols = [1,7,9])
df1.columns = ["Time", "lat", "lon"]
df1["Time"] = df1["Time"].apply(convert_time)
print(df1)

file2 = "/Users/Indhuja/Desktop/tcsp/lip/TCSP_LIP_fieldmill_2005.187_050706_valid.txt"
with open(file_fm, "r") as f:
    lines = f.readlines()
with open(file2, "w") as f:
    for line in lines:
        if ("NaN" not in line):
            f.write(line)
df2 = pd.read_csv(file2, sep=" ", header=None, skiprows=1, usecols=[0,1,2,3,4,5])
df2.columns = ['Date', 'Time', 'Ex', 'Ey', 'Ez', 'Eq']
df2["Time"] = df2["Time"].str.split('.').str[0]
print(df2)

merged_df = pd.merge(df1, df2, on="Time", how="inner")
print("merged df>>>>")
print(merged_df)

df = merged_df.groupby(['Time', 'Date'], as_index=False).agg({'Ex': 'mean', 'Ey': 'mean', 'Ez': 'mean', 'Eq': 'mean',
                                                        'lat': 'mean', 'lon': 'mean'})

tform1 = '%Y-%m-%d %H:%M:%S'
tform2 = '%Y-%m-%dT%H:%M:%SZ'
df['time2'] = [(datetime.strptime(d + ' ' + h, tform1) +
                timedelta(seconds=60)).strftime(tform2)
                for d, h in zip(df['Date'], df['Time'])] 

# df = df[(np.abs(df['Ex']) > 0.05) & (np.abs(df['Ey']) > 0.05) &(np.abs(df['Ez']) > 0.05) ]
df = df.reset_index(drop=True)
print(df)

czmlBody = [{"id": "document",
                "name": "LIP",
                "version": "1.0", }]

LIP = df[['Date', 'Time', 'time2',
            'Ex', 'Ey', 'Ez', 'lat', 'lon']]
print(LIP)

for d, t, t2, ex, ey, ez, lat, lon in zip(LIP.Date, LIP.Time, LIP.time2,
                                            LIP.Ex, LIP.Ey, LIP.Ez,
                                            LIP.lat, LIP.lon):
    xb = ex * .05
    yb = ey * .05
    zb = ez * 2000
    packet = {
        'id': t,
        'availability': d + 'T' + t + 'Z/' + t2,
        'polyline': {
            'positions': {'cartographicDegrees': [lon, lat, 0,
                                                    lon + xb, lat + yb, 0 + zb]},
            'material': {
                'polylineArrow': {
                    'color': {'rgba': [255, 55, 55, 255], }, }, },
            'width': 5}
    }

    czmlBody.append(packet)

LIPczml = json.dumps(czmlBody)
filepath = "/Users/Indhuja/Desktop/tcsp/lipcond/LIP0607.czml"
CZMLfile = open(filepath, "w")
CZMLfile.write(LIPczml)
CZMLfile.close()

           Time      lat       lon
0      19:51:21  9.99310 -84.21089
1      19:51:22  9.99310 -84.21089
2      19:51:23  9.99310 -84.21089
3      19:51:24  9.99310 -84.21089
4      19:51:25  9.99310 -84.21089
...         ...      ...       ...
37213  04:02:27  9.97598 -84.25591
37214  04:02:30  9.97654 -84.25439
37215  04:02:31  9.97673 -84.25382
37216  04:02:33  9.97691 -84.25340
37217  04:02:34  9.97710 -84.25288

[37218 rows x 3 columns]
             Date      Time        Ex        Ey        Ez        Eq
0      2005-07-06  19:57:01 -0.083008 -0.052244 -0.697938 -0.257670
1      2005-07-06  19:57:02 -0.058102  0.014377 -0.649696 -0.232297
2      2005-07-06  19:57:03 -0.057926  0.092869 -0.594393 -0.185313
3      2005-07-06  19:57:04 -0.065585 -0.014305 -0.660906 -0.249550
4      2005-07-06  19:57:05 -0.053211 -0.064829 -0.710588 -0.290402
...           ...       ...       ...       ...       ...       ...
28824  2005-07-07  04:07:48  0.226121 -0.082865 -0.678473 -0.422484
28825  200